In [ ]:
python manage.py runserver
.\venv\Scripts\activate

In [ ]:
import pandas as pd
#list of useful imports that  I will use
%matplotlib inline
import os

import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from glob import glob
import seaborn as sns
import random
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

In [ ]:
data = pd.read_csv(r'C:\Users\RAJA KANNAN\Music\PARKINSON\DATASET\parkinsons.csv')

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.isnull().any()

In [ ]:
data.drop('name',axis=1,inplace=True)

In [ ]:
data.corr()

In [ ]:
data['status'].value_counts()

In [ ]:
#counts of top 10 drugs
sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.countplot(x="status", data=data, palette=sns.color_palette("cubehelix", 4))
plt.xticks(rotation=90)
plt.title("Class Label Counts", {"fontname":"fantasy", "fontweight":"bold", "fontsize":"medium"})
plt.ylabel("count", {"fontname": "serif", "fontweight":"bold"})
plt.xlabel("Class", {"fontname": "serif", "fontweight":"bold"}) 

In [ ]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = data[data['status']== 1]
df_minority = data[data['status']== 0]
 
# Downsample majority class and upsample the minority class
df_minority_upsampled = resample(df_minority, replace=True,n_samples=1000,random_state=100) 
df_majority_downsampled = resample(df_majority, replace=True,n_samples=1000,random_state=100) 
 
# Combine minority class with downsampled majority class
df_balanced = pd.concat([df_minority_upsampled, df_majority_downsampled])
 
# Display new class counts
df_balanced['status'].value_counts()

In [ ]:
sns.countplot(df_balanced[['status']])
plt.grid()
plt.legend()
plt.title(' 0 :not affected & 1 : affected ')
plt.show()
print(' ')
plt.pie([1000,1000],labels=['not affected','affect'],autopct='%.2f%%')
plt.legend(loc=(1,0.5))
plt.title(' 0 :not affected & 1 : affect ')
plt.show()

In [ ]:
# shuffle the DataFrame rows 
data1= df_balanced.sample(frac = 1)

In [ ]:
data1

In [ ]:
data1.isnull().sum()

In [ ]:
data1.dropna(inplace=True)

In [ ]:
data

In [ ]:
#  get the all features except "status"
x = data.loc[:, data.columns != 'status']


In [ ]:
x

In [ ]:
y=data.iloc[:,-7]


In [ ]:
y

In [ ]:
x.head()

In [ ]:
y.tail()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,stratify=y ,random_state=40)

In [ ]:
x_train

In [ ]:
y_test

In [ ]:
#x_test.to_csv('Parkinsons_test.csv',index = False)

## SUPPORT VECTOR MACHINE

In [ ]:
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import math
from sklearn.metrics import accuracy_score


train_auc = []
cv_auc = []


model = SVC() 
#clf = CalibratedClassifierCV(model, cv=8)
model.fit(x_train,y_train)
prob_cv = model.predict(x_test)
cv_auc.append(accuracy_score(y_test,prob_cv))
prob_train = model.predict(x_train)
train_auc.append(accuracy_score(y_train,prob_train))
#optimal_C= C[cv_auc.index(max(cv_auc))]
#C=[math.log(x) for x in C]

#plot auc vs alpha
# x = plt.subplot( )
# x.plot(C, train_auc, label='AUC train')
# x.plot(C, cv_auc, label='AUC CV')
# plt.title('AUC vs hyperparameter')
# plt.xlabel('C')
# plt.ylabel('AUC')
# x.legend()
# plt.show()

#print('optimal C for which auc is maximum : ',optimal_C)

In [ ]:
#gamma = [10,20,30,30,40]

In [ ]:
#Testing AUC on Test data
svc = SVC()

svc.fit(x_train,y_train)
filename = r'C:\Users\RAJA KANNAN\Music\PARKINSON\CODING\frontend\svc_park.pkl'
pickle.dump(svc, open(filename, 'wb'))
#predict on test data and train data
 
y_predtests = svc.predict(x_test)
y_predtrains = svc.predict(x_train)

print('*'*35)

#accuracy on training and testing data

print('the accuracy on testing data',accuracy_score(y_test,y_predtests))
print('the accuracy on training data',accuracy_score(y_train,y_predtrains))
train = accuracy_score(y_train,y_predtrains)
test = accuracy_score(y_test,y_predtests)

print('*'*35)


# Code for drawing seaborn heatmaps
class_names = ['not affcted','affect']
cm = pd.DataFrame(confusion_matrix(y_test, y_predtests.round()), index=class_names, columns=class_names )
fig = plt.figure( )
heatmap = sns.heatmap(cm, annot=True, fmt="d")

In [ ]:
original =  ['affected' if x==1 else 'not affected' for x in y_test[:20]]
predicted = svc.predict(x_test[:20])
pred = []

for i in predicted:
  if i == 1:
    k = 'affected'
    pred.append(k)
  else:
    k = 'not affected'
    pred.append(k)
# Creating a data frame
dfr = pd.DataFrame(list(zip(original, pred,)), 
               columns =['original_Classlabel', 'predicted_classlebel'])
dfr

In [ ]:
all_model_result = pd.DataFrame(columns=[ 'Classifier' , 'Train-Accuracy', 'Test-Accuracy' ])
new = ['SUPPORT VECTOR-Classifier',train, test]
all_model_result.loc[0] = new

In [ ]:
import xgboost as xgb

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import GridSearchCV

dept = [1, 5, 10, 50, 100, 500, 1000]
n_estimators =  [20, 40, 60, 80, 100, 120]
xgb = XGBClassifier()

param_grid={'n_estimators':n_estimators , 'max_depth':dept}
model = GridSearchCV(xgb,param_grid,scoring='accuracy',n_jobs=-1,cv=3)
model.fit(x_train,y_train)
print("optimal n_estimators",model.best_estimator_.n_estimators)
print("optimal max_depth",model.best_estimator_.max_depth)
optimal_n_estimators = model.best_estimator_.n_estimators
optimal_max_depth = model.best_estimator_.max_depth

In [ ]:
#Testing AUC on Test data


xgb.fit(x_train,y_train)
filename = r'C:\Users\RAJA KANNAN\Music\PARKINSON\CODING\frontend\xgb_park.pkl'
pickle.dump(xgb, open(filename, 'wb'))
#predict on test data and train data
 
y_predtest = xgb.predict(x_test)
y_predtrain = xgb.predict(x_train)

print('*'*35)

#accuracy on training and testing data

print('the accuracy on testing data',accuracy_score(y_test,y_predtest))
print('the accuracy on training data',accuracy_score(y_train,y_predtrain))
train2 = accuracy_score(y_train,y_predtrain)
test2 = accuracy_score(y_test,y_predtest)

print('*'*35)


# Code for drawing seaborn heatmaps
class_names =  ['not affected','affected']
cm = pd.DataFrame(confusion_matrix(y_test, y_predtest.round()), index=class_names, columns=class_names )
fig = plt.figure( )
heatmap = sns.heatmap(cm, annot=True, fmt="d")

In [ ]:
original =  ['affected' if x==1 else 'not affected' for x in y_test[:20]]
predicted = xgb.predict(x_test[:20])
pred = []

for i in predicted:
  if i == 1:
    k = 'affected'
    pred.append(k)
  else:
    k = 'not affected'
    pred.append(k)
# Creating a data frame
dfr = pd.DataFrame(list(zip(original, pred,)), 
               columns =['original_Classlabel', 'predicted_classlebel'])
dfr

In [ ]:
new = ['XGB-Classifier',train2, test2]
all_model_result.loc[1] = new

In [ ]:
all_model_result